In [1]:
import pandas as pd
import random
import math
import numpy as np
import os

In [2]:
'''
Ce code permet d'étudier toute la base d'enquête. Puis, il procède à la jointure des tables ménage, personne et déplacement.
Il fait ensuite le mapping X,Y ; les départements ; les communes en fonction des codes INSEE comme clé de mapping. 
Ces mappings permettront de faire le filtre des périmètres d'étude (hauts-de-France et Nord-Pas-de-Calais). 
Le code prend en entrée les données issues du code dataHdF_preparing.ipynb stockés dans le dossier input_hdf et du 
traitement sur QGis qui a permis d'avoir pour chaque communes de la France ses coorodnnées X,Y, les codes des départements 
et de communes. Ce code renvoie dans output_hdf les données nécessaires à la génération de la demande de transport.
'''

"\nCe code permet d'étudier toute la base d'enquête. Puis, il procède à la jointure des tables ménage, personne et déplacement.\nIl fait ensuite le mapping X,Y ; les départements ; les communes en fonction des codes INSEE comme clé de mapping. \nCes mappings permettront de faire le filtre des périmètres d'étude (hauts-de-France et Nord-Pas-de-Calais). \nLe code prend en entrée les données issues du code dataHdF_preparing.ipynb stockés dans le dossier input_hdf et du \ntraitement sur QGis qui a permis d'avoir pour chaque communes de la France ses coorodnnées X,Y, les codes des départements \net de communes. Ce code renvoie dans output_hdf les données nécessaires à la génération de la demande de transport.\n"

In [14]:
print('=======> Read survey concatenated from dataHdF_preparing code and XY commune file from QGIS')
# Ces fonctions permettent d'indiquer les chemins des fichiers input
path_france_xy = "input_hdf/communesFranceXY.csv"
path_menage = "input_hdf/menage_hdf.csv"
path_person = "input_hdf/person_hdf.csv"
path_deplacement ="input_hdf/deplacement_hdf.csv"
path_trajet = "input_hdf/trajet_hdf.csv"
# chargement base donnée revenus disponibles (feuille Ensemble)
path_filosofi = "input_hdf/FILO_DISP_COM.xls"

=======> Read survey concatenated from dataHdF_preparing code and XY commune file from QGIS


In [15]:
# Cette fonction permet de lire données dans un fichier
#Elle prend en entrée le chemin du fichier
def read_file(file_path):
    df = pd.read_csv(file_path, sep=";",encoding='utf8')
    return df

In [16]:
#Cette fonction permet de sauvegarder des données dans un fichier
#Elle prend en entrée les données et le chemin du fichier de sortie

def save_file(df, output_path):
    df.to_csv(output_path,index=None, sep=',',encoding='utf8')

# Input

In [17]:
menage_hdf = pd.read_csv(path_menage, sep=";", dtype=str)
person_hdf  = pd.read_csv(path_person, sep=";", dtype=str)
deplacement_hdf = pd.read_csv(path_deplacement, sep=";", dtype=str)
trajet_hdf = pd.read_csv(path_menage, sep=";", dtype=str)
data_france_xy = pd.read_csv(path_france_xy, sep=",", dtype=str)
filosofi = pd.read_excel(path_filosofi, sheet_name = "ENSEMBLE", skiprows = 5, encoding='utf8', dtype=str)

In [18]:
#Compte le nombre de ménages, de personnes et de déplacements en Hauts de France de l'échantillon
nbrTotalMen_hdf=menage_hdf.ECH.count()
nbrTotalPers_hdf=person_hdf.PECH.count()
nbrTotalDepl_hdf=deplacement_hdf.DECH.count()

In [19]:
#Affiche le nombre de ménages, de personnes et de déplacements en Hauts de France de l'échantillon
print("-------> Found %d househols from menage.txt" % nbrTotalMen_hdf)
print("-------> Found %d persons from person.txt" % nbrTotalPers_hdf)
print("-------> Found %d trips from deplacement.txt" % nbrTotalDepl_hdf)

-------> Found 78251 househols from menage.txt
-------> Found 76751 persons from person.txt
-------> Found 301760 trips from deplacement.txt


# REVENU DES MENAGE EN HDF

In [22]:
print("-------> Affectation de revenu à chaque ménage")
filosofi

-------> Affectation de revenu à chaque ménage


,CODGEO,LIBGEO,NBMEN15,NBPERS15,NBUC15,Q115,Q215,Q315,Q3_Q1,D115,...,PCHO15,PTSA15,PBEN15,PPEN15,PPAT15,PPSOC15,PPFAM15,PPMINI15,PPLOGT15,PIMPOT15
0,01001,L'Abergement-Clémenciat,308,801,527.1,NaN,22228,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01002,L'Abergement-de-Varey,100,245.5,162.45,NaN,22883.33333333333,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01004,Ambérieu-en-Bugey,6216,13917,9543.70000000008,14314,19735.2,26194.666666666668,11880.666666666668,10398.571428571428,...,3.3,65.6,3.9,26.9,9.5,6.7,2.7,2,2,-15.8
3,01005,Ambérieux-en-Dombes,624,1671.5,1081.95,NaN,23182.666666666668,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01007,Ambronay,1044,2635.5,1732.7499999999916,17564,21986.5,27836.666666666668,10272.666666666668,13620.666666666666,...,2.5,71.2,2.6,26.7,8.1,4,2.4,0.8,0.8,-15.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32247,97420,Sainte-Suzanne,7706,23243.5,14391.149999999829,9980.8,14622.5,22349,12368.2,7684,...,4.8,66.6,7.5,11.6,4.9,16.1,5,7.6,3.4,-11.6
32248,97421,Salazie,2293,6880,4274.60000000002,8075.664251207731,11019.291666666668,15384.333333333332,7308.669082125602,5712.5,...,9,45.9,10.6,14.6,4.2,23.6,6.2,14.5,2.9,-7.8
32249,97422,Le Tampon,27310,73310,47091.80000000008,9646.875,13798.871635610763,21501.357142857145,11854.482142857145,7068.518518518518,...,5.3,53.6,7.8,20.3,8.1,16.6,4.5,8.1,4,-11.7
32250,97423,Les Trois-Bassins,2312,7024.5,4375.050000000027,9809.333333333334,13693.5,19586.153846153844,9776.82051282051,7242.857142857143,...,7.3,60.8,7.2,13.8,4.3,16.7,5,9.5,2.2,-10.1


In [46]:
'''Cette fonction permet d'affecter à chaque ménage de résidence MP2 en codes INSEE, des revenus
    Elle renvoie en sortie les nouvelle données avec les revnus
    Elle prend comme entrées: menage, data_income et output_file
        data est les données avec les code INSEE (menage.csv),
        filosofi les données de correspondance code CODGEO et (revenus),
        output_file est le chemin complet du fichier de sortie
'''
# Remplacement des codes INSEE par leur correspondance
def mapping_codeINSEE_income(data, filosofi):
    #select columns
    filosofi.columns =["CODGEO","LIBGEO","NBMEN15","NBPERS15","NBUC15","Q115","Q215","Q315","Q3_Q1","D115","D215","D315",
                      "D415","D615","D715","D815","D915","RD","S80S2015","GI15","PACT15","PCHO15","PTSA15","PBEN15",
                      "PPEN15","PPAT15","PPSOC15","PPFAM15","PPMINI15","PPLOGT15","PIMPOT15"]

# mapping
    # mapping code INSEE avec LIBGEO
    map_LIBGEO = dict(zip(filosofi.CODGEO, filosofi.LIBGEO))
    # mapping code INSEE avec NBMEN15
    map_NBMEN15 = dict(zip(filosofi.CODGEO, filosofi.NBMEN15))
    # mapping code INSEE avec NBPERS15
    map_NBPERS15 = dict(zip(filosofi.CODGEO, filosofi.NBPERS15))
    # mapping code INSEE avec NBUC15
    map_NBUC15 = dict(zip(filosofi.CODGEO, filosofi.NBUC15))
    # mapping code INSEE avec Q115
    map_Q115 = dict(zip(filosofi.CODGEO, filosofi.Q115))
    # mapping code INSEE avec Q215
    map_Q215 = dict(zip(filosofi.CODGEO, filosofi.Q215))
    # mapping code INSEE avec Q315
    map_Q315 = dict(zip(filosofi.CODGEO, filosofi.Q315))
    # mapping code INSEE avec Q3_Q1
    map_Q3_Q1 = dict(zip(filosofi.CODGEO, filosofi.Q3_Q1))
    # mapping code INSEE avec D115
    map_D115 = dict(zip(filosofi.CODGEO, filosofi.D115))
    # mapping code INSEE avec D215
    map_D215 = dict(zip(filosofi.CODGEO, filosofi.D215))
    # mapping code INSEE avec D315
    map_D315 = dict(zip(filosofi.CODGEO, filosofi.D315))
    # mapping code INSEE avec D415
    map_D415 = dict(zip(filosofi.CODGEO, filosofi.D415))
    # mapping code INSEE avec D615
    map_D615 = dict(zip(filosofi.CODGEO, filosofi.D615))
    # mapping code INSEE avec D715
    map_D715 = dict(zip(filosofi.CODGEO, filosofi.D715))
    # mapping code INSEE avec D815
    map_D815 = dict(zip(filosofi.CODGEO, filosofi.D815))
    # mapping code INSEE avec D915
    map_D915 = dict(zip(filosofi.CODGEO, filosofi.D915))
    # mapping code INSEE avec RD
    map_RD = dict(zip(filosofi.CODGEO, filosofi.RD))
    # mapping code INSEE avec S80S2015
    map_S80S2015 = dict(zip(filosofi.CODGEO, filosofi.S80S2015))
    # mapping code INSEE avec GI15
    map_GI15 = dict(zip(filosofi.CODGEO, filosofi.GI15))
    # mapping code INSEE avec PACT15
    map_PACT15 = dict(zip(filosofi.CODGEO, filosofi.PACT15))
    # mapping code INSEE avec PCHO15
    map_PCHO15 = dict(zip(filosofi.CODGEO, filosofi.PCHO15))
    # mapping code INSEE avec PTSA15
    map_PTSA15 = dict(zip(filosofi.CODGEO, filosofi.PTSA15))
    # mapping code INSEE avec PBEN15
    map_PBEN15 = dict(zip(filosofi.CODGEO, filosofi.PBEN15))
    # mapping code INSEE avec PPEN15
    map_PPEN15 = dict(zip(filosofi.CODGEO, filosofi.PPEN15))
    # mapping code INSEE avec PPAT15
    map_PPAT15 = dict(zip(filosofi.CODGEO, filosofi.PPAT15))
    # mapping code INSEE avec PPSOC15
    map_PPSOC15 = dict(zip(filosofi.CODGEO, filosofi.PPSOC15))
    # mapping code INSEE avec PPFAM15
    map_PPFAM15 = dict(zip(filosofi.CODGEO, filosofi.PPFAM15))
    # mapping code INSEE avec PPMINI15
    map_PPMINI15 = dict(zip(filosofi.CODGEO, filosofi.PPMINI15))
    # mapping code INSEE avec PPLOGT15
    map_PPLOGT15 = dict(zip(filosofi.CODGEO, filosofi.PPLOGT15))
    # mapping code INSEE avec PIMPOT15
    map_PIMPOT15 = dict(zip(filosofi.CODGEO, filosofi.PIMPOT15))
    
# création des colonnes
    #Créer des colonnes pour les revenus des résidences des ménages
    data["LIBGEO"]=data["MP2"]
    data["NBMEN15"]=data["MP2"]
    data["NBPERS15"]=data["MP2"]
    data["NBUC15"]=data["MP2"]
    data["Q115"]=data["MP2"]
    data["Q215"]=data["MP2"]
    data["Q315"]=data["MP2"]
    data["Q3_Q1"]=data["MP2"]
    data["D115"]=data["MP2"]
    data["D215"]=data["MP2"]
    data["D315"]=data["MP2"]
    data["D415"]=data["MP2"]
    data["D615"]=data["MP2"]
    data["D715"]=data["MP2"]
    data["D815"]=data["MP2"]
    data["D915"]=data["MP2"]
    data["RD"]=data["MP2"]
    data["S80S2015"]=data["MP2"]
    data["GI15"]=data["MP2"]
    data["PACT15"]=data["MP2"]
    data["PCHO15"]=data["MP2"]
    data["PTSA15"]=data["MP2"]
    data["PBEN15"]=data["MP2"]
    data["PPEN15"]=data["MP2"]
    data["PPAT15"]=data["MP2"]
    data["PPSOC15"]=data["MP2"]
    data["PPFAM15"]=data["MP2"]
    data["PPMINI15"]=data["MP2"]
    data["PPLOGT15"]=data["MP2"]
    data["PIMPOT15"]=data["MP2"]
# remplacement 
    # remplacer les codes INSEE par les revenus
    data[["LIBGEO"]] = data[["LIBGEO"]].applymap(map_LIBGEO.get)
    data[["NBMEN15"]] = data[["NBMEN15"]].applymap(map_NBMEN15.get)
    data[["NBPERS15"]] = data[["NBPERS15"]].applymap(map_NBPERS15.get)
    data[["NBUC15"]] = data[["NBUC15"]].applymap(map_NBUC15.get)
    data[["Q115"]] = data[["Q115"]].applymap(map_Q115.get)
    data[["Q215"]] = data[["Q215"]].applymap(map_Q215.get)
    data[["Q315"]] = data[["Q315"]].applymap(map_Q315.get)
    data[["Q3_Q1"]] = data[["Q3_Q1"]].applymap(map_Q3_Q1.get)
    data[["D115"]] = data[["D115"]].applymap(map_D115.get)
    data[["D215"]] = data[["D215"]].applymap(map_D215.get)
    data[["D315"]] = data[["D315"]].applymap(map_D315.get)
    data[["D415"]] = data[["D415"]].applymap(map_D415.get)
    data[["D615"]] = data[["D615"]].applymap(map_D615.get)
    data[["D715"]] = data[["D715"]].applymap(map_D715.get)
    data[["D815"]] = data[["D815"]].applymap(map_D815.get)
    data[["D915"]] = data[["D915"]].applymap(map_D915.get)
    data[["RD"]] = data[["RD"]].applymap(map_RD.get)
    data[["S80S2015"]] = data[["S80S2015"]].applymap(map_S80S2015.get)
    data[["GI15"]] = data[["GI15"]].applymap(map_GI15.get)
    data[["PACT15"]] = data[["PACT15"]].applymap(map_PACT15.get)
    data[["PCHO15"]] = data[["PCHO15"]].applymap(map_PCHO15.get)
    data[["PTSA15"]] = data[["PTSA15"]].applymap(map_PTSA15.get)
    data[["PBEN15"]] = data[["PBEN15"]].applymap(map_PBEN15.get)
    data[["PPEN15"]] = data[["PPEN15"]].applymap(map_PPEN15.get)
    data[["PPAT15"]] = data[["PPAT15"]].applymap(map_PPAT15.get)
    data[["PPSOC15"]] = data[["PPSOC15"]].applymap(map_PPSOC15.get)
    data[["PPFAM15"]] = data[["PPFAM15"]].applymap(map_PPFAM15.get)
    data[["PPMINI15"]] = data[["PPMINI15"]].applymap(map_PPMINI15.get)
    data[["PPLOGT15"]] = data[["PPLOGT15"]].applymap(map_PPLOGT15.get)
    data[["PIMPOT15"]] = data[["PIMPOT15"]].applymap(map_PIMPOT15.get)
    
# suppression des données dont les ménages ne sont pas dans les HdF ==> menage_income    
    data = data.dropna(subset=["Q215"])
    return data

In [47]:
# Application de la fonction de mapping MP2 avec les revenus : 
# 119 ménages n'ont pas de revenus et devront prendre les valeurs des ménages plus proches
menage_income = mapping_codeINSEE_income(menage_hdf, filosofi)

In [48]:
NbrCommSansRevMedian = len(menage_hdf) - len(menage_income)
print('-------> Nombre de ménages sans revenu médian =')
print(NbrCommSansRevMedian)

-------> Nombre de ménages sans revenu médian =
423


In [49]:
menage_hdf

,MP1,MEMD,MANN,ECH,MP2,MTIR,MTLO,M1,M2,M3D,...,PCHO15,PTSA15,PBEN15,PPEN15,PPAT15,PPSOC15,PPFAM15,PPMINI15,PPLOGT15,PIMPOT15
0,1,01,2004,012004000114184,62119,01,000114,3,3,1,...,3.5,55.9,7.9,28.9,10.6,10.8,2.8,4.5,3.6,-17.6
1,1,01,2004,012004000116063,62119,01,000116,3,3,2,...,3.5,55.9,7.9,28.9,10.6,10.8,2.8,4.5,3.6,-17.6
2,1,01,2004,012004000116095,62119,01,000116,4,1,1,...,3.5,55.9,7.9,28.9,10.6,10.8,2.8,4.5,3.6,-17.6
3,1,01,2004,012004000116162,62119,01,000116,2,3,1,...,3.5,55.9,7.9,28.9,10.6,10.8,2.8,4.5,3.6,-17.6
4,1,01,2004,012004000117090,62119,01,000117,4,2,2,...,3.5,55.9,7.9,28.9,10.6,10.8,2.8,4.5,3.6,-17.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78246,1,30,2014,3020144560093556,80021,46,456009,NaN,NaN,NaN,...,3.6,62,6.2,26.4,9,9.9,2.7,4,3.2,-17.1
78247,1,30,2014,3020144560093566,80021,46,456009,NaN,NaN,NaN,...,3.6,62,6.2,26.4,9,9.9,2.7,4,3.2,-17.1
78248,1,30,2014,3020144560093585,80021,46,456009,NaN,NaN,NaN,...,3.6,62,6.2,26.4,9,9.9,2.7,4,3.2,-17.1
78249,1,30,2014,3020144560093629,80021,46,456009,NaN,NaN,1,...,3.6,62,6.2,26.4,9,9.9,2.7,4,3.2,-17.1


# DEPLACEMENT DES PERSONNES EN HdF

In [50]:
print('=======> Merge menage and person')
# Quels sont les ménages des personnes
menage_person_hdf = pd.merge(person_hdf, menage_hdf, how='left',on ="ECH")

=======> Merge menage and person


In [51]:
print('--------> menage_person_hdf')
menage_person_hdf

--------> menage_person_hdf


,PP1,PEMD,PANN,ECH,PECH,PP2,PTIR,PTLO,PER,P2,...,PCHO15,PTSA15,PBEN15,PPEN15,PPAT15,PPSOC15,PPFAM15,PPMINI15,PPLOGT15,PIMPOT15
0,2,01,2004,012004000118174,01200400011817402,62119,01,000118,02,2,...,3.5,55.9,7.9,28.9,10.6,10.8,2.8,4.5,3.6,-17.6
1,2,01,2004,012004000119131,01200400011913101,62119,01,000119,01,2,...,3.5,55.9,7.9,28.9,10.6,10.8,2.8,4.5,3.6,-17.6
2,2,01,2004,012004000119131,01200400011913102,62119,01,000119,02,1,...,3.5,55.9,7.9,28.9,10.6,10.8,2.8,4.5,3.6,-17.6
3,2,01,2004,012004000119185,01200400011918501,62119,01,000119,01,2,...,3.5,55.9,7.9,28.9,10.6,10.8,2.8,4.5,3.6,-17.6
4,2,01,2004,012004000120149,01200400012014901,62119,01,000120,01,2,...,3.5,55.9,7.9,28.9,10.6,10.8,2.8,4.5,3.6,-17.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76746,2,30,2014,3020144560092582,302014456009258202,80021,46,456009,02,2,...,3.6,62,6.2,26.4,9,9.9,2.7,4,3.2,-17.1
76747,2,30,2014,3020144560092921,302014456009292102,80021,46,456009,02,2,...,3.6,62,6.2,26.4,9,9.9,2.7,4,3.2,-17.1
76748,2,30,2014,3020144560093087,302014456009308701,80021,46,456009,01,2,...,3.6,62,6.2,26.4,9,9.9,2.7,4,3.2,-17.1
76749,2,30,2014,3020144560093629,302014456009362901,80021,46,456009,01,2,...,3.6,62,6.2,26.4,9,9.9,2.7,4,3.2,-17.1


In [52]:
menage_person_hdf.to_csv("output_hdf/fusion/menage_person_hdf.csv", index=None, sep=";")

In [53]:
print('=======> Merge menage, person and deplacement')

=======> Merge menage, person and deplacement


In [54]:
# Quels sont les déplacements des personnes : 
deplacement_menage_person_hdf = pd.merge(deplacement_hdf, menage_person_hdf, how='left',on ="PECH")

In [55]:
# Trier selon DECH
trips=deplacement_menage_person_hdf.sort_values(by = 'DECH')

In [56]:
trips.to_csv("output_hdf/fusion/trips.csv", index=None, sep=";")

In [57]:
print('--------> deplacement_menage_person_hdf')
trips

--------> deplacement_menage_person_hdf


,DP1,DEMD,DANN,ECH_x,PECH,DECH,DP2,DTIR,DTLO,PER_x,...,PCHO15,PTSA15,PBEN15,PPEN15,PPAT15,PPSOC15,PPFAM15,PPMINI15,PPLOGT15,PIMPOT15
20,3,01,2004,012004000101004,01200400010100401,0120040001010040101,62119,01,000101,01,...,3.5,55.9,7.9,28.9,10.6,10.8,2.8,4.5,3.6,-17.6
21,3,01,2004,012004000101004,01200400010100401,0120040001010040102,62119,01,000101,01,...,3.5,55.9,7.9,28.9,10.6,10.8,2.8,4.5,3.6,-17.6
22,3,01,2004,012004000101004,01200400010100402,0120040001010040201,62119,01,000101,02,...,3.5,55.9,7.9,28.9,10.6,10.8,2.8,4.5,3.6,-17.6
23,3,01,2004,012004000101004,01200400010100402,0120040001010040202,62119,01,000101,02,...,3.5,55.9,7.9,28.9,10.6,10.8,2.8,4.5,3.6,-17.6
24,3,01,2004,012004000101004,01200400010100402,0120040001010040203,62119,01,000101,02,...,3.5,55.9,7.9,28.9,10.6,10.8,2.8,4.5,3.6,-17.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301755,3,30,2014,3020144560093087,302014456009308701,30201445600930870103,80021,46,456009,01,...,3.6,62,6.2,26.4,9,9.9,2.7,4,3.2,-17.1
301756,3,30,2014,3020144560093629,302014456009362901,30201445600936290101,80021,46,456009,01,...,3.6,62,6.2,26.4,9,9.9,2.7,4,3.2,-17.1
301757,3,30,2014,3020144560093629,302014456009362901,30201445600936290102,80021,46,456009,01,...,3.6,62,6.2,26.4,9,9.9,2.7,4,3.2,-17.1
301758,3,30,2014,3020144560093672,302014456009367201,30201445600936720101,80021,46,456009,01,...,3.6,62,6.2,26.4,9,9.9,2.7,4,3.2,-17.1


# Jointure des données pour récuperer les coordonnées X et Y

In [58]:
print('=========> Merge trips with cordonates XY')
print('---------> Base XY centroid of municiplities of France')
data_france_xy

=========> Merge trips with cordonates XY
---------> Base XY centroid of municiplities of France


,ID_GEOFLA,CODE_COM,INSEE_CODE,NOM_COM,STATUT,X_CHF_LIEU,Y_CHF_LIEU,X_CENTROID,Y_CENTROID,Z_MOYEN,SUPERFICIE,POPULATION,CODE_CANT,CODE_ARR,CODE_DEPT,NOM_DEPT,CODE_REG,NOM_REG,Xcord,Ycord
0,COMMUNE00000000000016437,062,60062,BERLANCOURT,Commune simple,704984,6952471,706330,6953686,85,711,336,17,3,60,OISE,32,NORD-PAS-DE-CALAIS-PICARDIE,706060.90944,6953310.14567
1,COMMUNE00000000000001650,238,77238,JOUARRE,Commune simple,709517,6869668,708647,6867909,145,4237,4180,06,1,77,SEINE-ET-MARNE,11,ILE-DE-FRANCE,708255.88425,6866954.50197
2,COMMUNE00000000000002116,311,77311,MONTIGNY-LENCOUP,Commune simple,704721,6816965,703819,6817676,120,2078,1284,18,3,77,SEINE-ET-MARNE,11,ILE-DE-FRANCE,703862.50406,6817112.36431
3,COMMUNE00000000000002657,138,89138,DICY,Commune simple,707969,6759226,707162,6758983,159,1018,329,08,1,89,YONNE,27,BOURGOGNE-FRANCHE-COMTE,707072.36065,6758587.60754
4,COMMUNE00000000000003749,133,77133,COURCELLES-EN-BASSEE,Commune simple,704064,6812147,704030,6811923,90,1082,239,13,3,77,SEINE-ET-MARNE,11,ILE-DE-FRANCE,703917.70779,6812513.21925
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36566,COMMUNE00000000000036527,419,61419,SAINTE-MARGUERITE-DE-CARROUGES,Commune simple,467426,6836201,467678,6837481,247,863,223,15,1,61,ORNE,28,NORMANDIE,467671.71651,6837193.67968
36567,COMMUNE00000000000000579,194,86194,POITIERS,Pr?fecture de d?partement,495982,6612457,496930,6611079,112,4242,87646,99,3,86,VIENNE,75,AQUITAINE-LIMOUSIN-POITOU-CHARENTES,497843.05684,6612644.45115
36568,COMMUNE00000000000014929,085,87085,LIMOGES,Pr?fecture de d?partement,565065,6527578,562539,6527335,310,7971,136221,99,2,87,HAUTE-VIENNE,75,AQUITAINE-LIMOUSIN-POITOU-CHARENTES,564132.69639,6529813.24768
36569,COMMUNE00000000000032196,056,25056,BESANCON,Pr?fecture de d?partement,928499,6686815,927844,6686326,324,6510,116353,99,1,25,DOUBS,27,BOURGOGNE-FRANCHE-COMTE,928283.92969,6688232.57611


In [59]:
'''Cette fonction permet de remplacer les codes INSEE par les coordonnées (X et Y), les départements
    Elle renvoie en sortie les nouvelle données avec les coordonnées X et Y, les codes départementaux 
    Elle prend comme entrées: data, data_france_xy et output_file
        data est les données avec les code INSEE (déplacement.txt),
        data_france_xy les données de correspondance code INSEE et (X, Y) des communes françaises,
        output_file est le chemin complet du fichier de sortie
'''

# Remplacement des codes INSEE par leur correspondance
def mapping_codeINSEE_xyFrance(data, data_france_xy):
    #select columns
    data_france_xy.columns =["ID_GEOFLA","CODE_COM","INSEE_CODE","NOM_COM","STATUT","X_CHF_LIEU","Y_CHF_LIEU","X_CENTROID",
                             "Y_CENTROID","Z_MOYEN","SUPERFICIE","POPULATION","CODE_CANT","CODE_ARR","CODE_DEPT","NOM_DEPT",
                             "CODE_REG","NOM_REG","Xcord","Ycord"]
# mapping

    # mapping code INSEE et coordonnées (nécessaire pour la localisation des activités : pour les variables D3 et D7)
    map_x = dict(zip(data_france_xy.INSEE_CODE, data_france_xy.Xcord))
    map_y = dict(zip(data_france_xy.INSEE_CODE, data_france_xy.Ycord))
   
    # mapping code INSEE avec superficie (nécessaire pour la distribution des agents de la même commune : pour D3 et D7)
    map_sup = dict(zip(data_france_xy.INSEE_CODE, data_france_xy.SUPERFICIE))
    
    # mapping code INSEE avec département (nécssaire pour l'anayse des flux : pour les variables PP2, DP2, D3 et D7)
    map_dept = dict(zip(data_france_xy.INSEE_CODE, data_france_xy.CODE_DEPT))
    
    # mapping code INSEE avec région (nécssaire pour l'anayse des flux : pour les variables PP2, DP2, D3 et D7)
    map_reg = dict(zip(data_france_xy.INSEE_CODE, data_france_xy.CODE_REG))

    
# création de colonnes

    #Créer des colonnes pour X et Y
    data["D3X"] = data["D3"]
    data["D3Y"] = data["D3"]
    data["D7X"] = data["D7"]
    data["D7Y"] = data["D7"]
    
    # Créer des colonnes pour Superficie
    data["SUP_D3"]=data["D3"]
    data["SUP_D7"]=data["D7"]
    
    # créer des colonnes pour CODE_DEPT
    data["CODEDEPT_D3"]=data["D3"]
    data["CODEDEPT_D7"]=data["D7"]
    data["CODEDEPT_PP2"]=data["PP2"]
    data["CODEDEPT_DP2"]=data["DP2"]
    
    # créer des colonnes pour CODE_REG
    data["CODEREG_D3"]=data["D3"]
    data["CODEREG_D7"]=data["D7"]
    data["CODEREG_PP2"]=data["PP2"]
    data["CODEREG_DP2"]=data["DP2"]

        
# remplacement    

    # remplacer les codes INSEE par les coordonnées
    data[["D3X","D7X"]] = data[["D3X","D7X"]].applymap(map_x.get)
    data[["D3Y","D7Y"]] = data[["D3Y","D7Y"]].applymap(map_y.get)
    
    # remplacer les codes INSEE par les superficies
    data[["SUP_D3"]] = data[["SUP_D3"]].applymap(map_sup.get)
    data[["SUP_D7"]] = data[["SUP_D7"]].applymap(map_sup.get)
    
    # remplacer les codes INSEE par les départements
    data[["CODEDEPT_D3"]] = data[["CODEDEPT_D3"]].applymap(map_dept.get)
    data[["CODEDEPT_D7"]] = data[["CODEDEPT_D7"]].applymap(map_dept.get)
    data[["CODEDEPT_PP2"]] = data[["CODEDEPT_PP2"]].applymap(map_dept.get)
    data[["CODEDEPT_DP2"]] = data[["CODEDEPT_DP2"]].applymap(map_dept.get)

    # remplacer les codes INSEE par les régions
    data[["CODEREG_D3"]] = data[["CODEREG_D3"]].applymap(map_reg.get)
    data[["CODEREG_D7"]] = data[["CODEREG_D7"]].applymap(map_reg.get)
    data[["CODEREG_PP2"]] = data[["CODEREG_PP2"]].applymap(map_reg.get)
    data[["CODEREG_DP2"]] = data[["CODEREG_DP2"]].applymap(map_reg.get)

    
    # suppression des données dont les coordonnées X,Y ne figurent pas dans la base ComunesXY.csv ==> travelDiary_hdf
    data = data.dropna(subset=["D3X","D3Y", "D7X","D7Y"])
    
    #data[cols].to_csv(output_file,index=None, sep=";")
    
    return data

In [60]:
# Application de la fonction de mapping à partir des XY de la france
print('---------> Application of mapping XY function')
deplacement_xy_france = mapping_codeINSEE_xyFrance(trips, data_france_xy)

---------> Application of mapping XY function


In [61]:
print('---------> deplacement_xy_france mapped')
deplacement_xy_france # 5310 déplacements sans X ou Y de la base HdF

---------> deplacement_xy_france mapped


,DP1,DEMD,DANN,ECH_x,PECH,DECH,DP2,DTIR,DTLO,PER_x,...,SUP_D3,SUP_D7,CODEDEPT_D3,CODEDEPT_D7,CODEDEPT_PP2,CODEDEPT_DP2,CODEREG_D3,CODEREG_D7,CODEREG_PP2,CODEREG_DP2
20,3,01,2004,012004000101004,01200400010100401,0120040001010040101,62119,01,000101,01,...,950,950,62,62,62,62,32,32,32,32
21,3,01,2004,012004000101004,01200400010100401,0120040001010040102,62119,01,000101,01,...,950,950,62,62,62,62,32,32,32,32
22,3,01,2004,012004000101004,01200400010100402,0120040001010040201,62119,01,000101,02,...,950,950,62,62,62,62,32,32,32,32
23,3,01,2004,012004000101004,01200400010100402,0120040001010040202,62119,01,000101,02,...,950,950,62,62,62,62,32,32,32,32
24,3,01,2004,012004000101004,01200400010100402,0120040001010040203,62119,01,000101,02,...,950,950,62,62,62,62,32,32,32,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301755,3,30,2014,3020144560093087,302014456009308701,30201445600930870103,80021,46,456009,01,...,5009,5009,80,80,80,80,32,32,32,32
301756,3,30,2014,3020144560093629,302014456009362901,30201445600936290101,80021,46,456009,01,...,5009,1562,80,80,80,80,32,32,32,32
301757,3,30,2014,3020144560093629,302014456009362901,30201445600936290102,80021,46,456009,01,...,1562,5009,80,80,80,80,32,32,32,32
301758,3,30,2014,3020144560093672,302014456009367201,30201445600936720101,80021,46,456009,01,...,5009,2366,80,80,80,80,32,32,32,32


In [62]:
print('---------> deplacement_xy_france mapped with origin or destination not in France')
trips 

---------> deplacement_xy_france mapped with origin or destination not in France


,DP1,DEMD,DANN,ECH_x,PECH,DECH,DP2,DTIR,DTLO,PER_x,...,SUP_D3,SUP_D7,CODEDEPT_D3,CODEDEPT_D7,CODEDEPT_PP2,CODEDEPT_DP2,CODEREG_D3,CODEREG_D7,CODEREG_PP2,CODEREG_DP2
20,3,01,2004,012004000101004,01200400010100401,0120040001010040101,62119,01,000101,01,...,950,950,62,62,62,62,32,32,32,32
21,3,01,2004,012004000101004,01200400010100401,0120040001010040102,62119,01,000101,01,...,950,950,62,62,62,62,32,32,32,32
22,3,01,2004,012004000101004,01200400010100402,0120040001010040201,62119,01,000101,02,...,950,950,62,62,62,62,32,32,32,32
23,3,01,2004,012004000101004,01200400010100402,0120040001010040202,62119,01,000101,02,...,950,950,62,62,62,62,32,32,32,32
24,3,01,2004,012004000101004,01200400010100402,0120040001010040203,62119,01,000101,02,...,950,950,62,62,62,62,32,32,32,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301755,3,30,2014,3020144560093087,302014456009308701,30201445600930870103,80021,46,456009,01,...,5009,5009,80,80,80,80,32,32,32,32
301756,3,30,2014,3020144560093629,302014456009362901,30201445600936290101,80021,46,456009,01,...,5009,1562,80,80,80,80,32,32,32,32
301757,3,30,2014,3020144560093629,302014456009362901,30201445600936290102,80021,46,456009,01,...,1562,5009,80,80,80,80,32,32,32,32
301758,3,30,2014,3020144560093672,302014456009367201,30201445600936720101,80021,46,456009,01,...,5009,2366,80,80,80,80,32,32,32,32


In [63]:
trips_outside = len(trips) - len(deplacement_xy_france)
print("-------> Found %d trips for which the origin or the destination is out of France" % trips_outside)

-------> Found 5310 trips for which the origin or the destination is out of France


In [77]:
print("========> Extraction of relevant variables for MATSim")
# Extraction des variables pertinentes pour l'étude : données qui n'ont pas de correspondance X,Y sont enlevées
travelDiary_hdf = deplacement_xy_france[["DECH","PECH","COEQ","DP2","D4A","D4B","D3","D3X","D3Y","D2AA","D8A","D8B",
                                         "D7","D7X","D7Y","D5AA","MODP","NDEP","SUP_D3","SUP_D7","CODEDEPT_D3","CODEREG_D3",
                                         "CODEDEPT_D7","CODEREG_D7","MTDQ","PTDQ","P4","P2","P5","P10","M5","DOIB","DIST",
                                         "D8C","D9","M6A","M6B","M6C","M6D","CODEDEPT_PP2","CODEREG_PP2","CODEDEPT_DP2",
                                         "CODEREG_DP2","Q115","Q215","Q315","Q3_Q1","D115","D215","D315","D415","D615",
                                         "D715","D815","D915","ECH_x"]]

========> Extraction of relevant variables for MATSim


In [78]:
travelDiary_hdf

,DECH,PECH,COEQ,DP2,D4A,D4B,D3,D3X,D3Y,D2AA,...,Q3_Q1,D115,D215,D315,D415,D615,D715,D815,D915,ECH_x
20,0120040001010040101,01200400010100401,36.8921544,62119,09,45,62119,674661.80202,7048040.28762,01,...,11899.166666666666,8858,10837,12729.62962962963,14623,19225.416666666668,21900.666666666668,25877.33333333333,33081.333333333336,012004000101004
21,0120040001010040102,01200400010100401,36.8921544,62119,10,30,62119,674661.80202,7048040.28762,06,...,11899.166666666666,8858,10837,12729.62962962963,14623,19225.416666666668,21900.666666666668,25877.33333333333,33081.333333333336,012004000101004
22,0120040001010040201,01200400010100402,36.8921544,62119,09,45,62119,674661.80202,7048040.28762,01,...,11899.166666666666,8858,10837,12729.62962962963,14623,19225.416666666668,21900.666666666668,25877.33333333333,33081.333333333336,012004000101004
23,0120040001010040202,01200400010100402,36.8921544,62119,10,30,62119,674661.80202,7048040.28762,06,...,11899.166666666666,8858,10837,12729.62962962963,14623,19225.416666666668,21900.666666666668,25877.33333333333,33081.333333333336,012004000101004
24,0120040001010040203,01200400010100402,36.8921544,62119,10,45,62119,674661.80202,7048040.28762,01,...,11899.166666666666,8858,10837,12729.62962962963,14623,19225.416666666668,21900.666666666668,25877.33333333333,33081.333333333336,012004000101004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301755,30201445600930870103,302014456009308701,8.93880200,80021,18,15,80021,649001.84505,6978182.20329,06,...,12510.304347826086,8911.724137931034,11167.560975609756,13250.5,15356.8,20084,22956,27043.33333333333,34545,3020144560093087
301756,30201445600936290101,302014456009362901,7.49811400,80021,18,00,80021,649001.84505,6978182.20329,01,...,12510.304347826086,8911.724137931034,11167.560975609756,13250.5,15356.8,20084,22956,27043.33333333333,34545,3020144560093629
301757,30201445600936290102,302014456009362901,7.49811400,80021,22,40,80685,684738.87069,6954986.49399,09,...,12510.304347826086,8911.724137931034,11167.560975609756,13250.5,15356.8,20084,22956,27043.33333333333,34545,3020144560093629
301758,30201445600936720101,302014456009367201,8.88031600,80021,08,30,80021,649001.84505,6978182.20329,01,...,12510.304347826086,8911.724137931034,11167.560975609756,13250.5,15356.8,20084,22956,27043.33333333333,34545,3020144560093672


In [79]:
travelDiary_hdf.to_csv("output_hdf/travelDiaries/travelDiary_hdf.csv", index=None, sep=";")

# Application du filtre pour le Nord pas de Calais

In [82]:
## Déplacements pour le Nord-Pas-de-calais en tant qu'origine ou destination de flux de la région
## travelDiary_hdf est utilisé car constitue la base où les X,Y des communes sont connus 
print("========> Filter trips in studied area : Departments of Nord-Pas-de-Calais has been taked as spatial zone")
is_deplNpdcXY = (travelDiary_hdf["CODEDEPT_D3"] == '59') | (travelDiary_hdf["CODEDEPT_D7"]=='62') | (travelDiary_hdf["CODEDEPT_D7"] == '59') | (travelDiary_hdf["CODEDEPT_D3"]=='62')

travelDiary_npdcXY = travelDiary_hdf[is_deplNpdcXY]

travelDiary_npdcXY.to_csv("output_hdf/travelDiaries/travelDiaryDeciles_npdcXY.csv", index=None, sep=";")

========> Filter trips in studied area : Departments of Nord-Pas-de-Calais has been taked as spatial zone


In [83]:
travelDiary_npdcXY

,DECH,PECH,COEQ,DP2,D4A,D4B,D3,D3X,D3Y,D2AA,...,Q3_Q1,D115,D215,D315,D415,D615,D715,D815,D915,ECH_x
20,0120040001010040101,01200400010100401,36.8921544,62119,09,45,62119,674661.80202,7048040.28762,01,...,11899.166666666666,8858,10837,12729.62962962963,14623,19225.416666666668,21900.666666666668,25877.33333333333,33081.333333333336,012004000101004
21,0120040001010040102,01200400010100401,36.8921544,62119,10,30,62119,674661.80202,7048040.28762,06,...,11899.166666666666,8858,10837,12729.62962962963,14623,19225.416666666668,21900.666666666668,25877.33333333333,33081.333333333336,012004000101004
22,0120040001010040201,01200400010100402,36.8921544,62119,09,45,62119,674661.80202,7048040.28762,01,...,11899.166666666666,8858,10837,12729.62962962963,14623,19225.416666666668,21900.666666666668,25877.33333333333,33081.333333333336,012004000101004
23,0120040001010040202,01200400010100402,36.8921544,62119,10,30,62119,674661.80202,7048040.28762,06,...,11899.166666666666,8858,10837,12729.62962962963,14623,19225.416666666668,21900.666666666668,25877.33333333333,33081.333333333336,012004000101004
24,0120040001010040203,01200400010100402,36.8921544,62119,10,45,62119,674661.80202,7048040.28762,01,...,11899.166666666666,8858,10837,12729.62962962963,14623,19225.416666666668,21900.666666666668,25877.33333333333,33081.333333333336,012004000101004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301604,30201445600633260102,302014456006332601,9.42639200,80021,16,15,62447,631483.46449,7031127.49468,02,...,12510.304347826086,8911.724137931034,11167.560975609756,13250.5,15356.8,20084,22956,27043.33333333333,34545,3020144560063326
301696,30201445600909400105,302014456009094001,8.06854600,80021,11,00,80021,649001.84505,6978182.20329,01,...,12510.304347826086,8911.724137931034,11167.560975609756,13250.5,15356.8,20084,22956,27043.33333333333,34545,3020144560090940
301697,30201445600909400106,302014456009094001,8.06854600,80021,19,30,62663,660095.34981,7007901.02731,09,...,12510.304347826086,8911.724137931034,11167.560975609756,13250.5,15356.8,20084,22956,27043.33333333333,34545,3020144560090940
301698,30201445600909470101,302014456009094701,8.94751700,80021,05,30,80021,649001.84505,6978182.20329,01,...,12510.304347826086,8911.724137931034,11167.560975609756,13250.5,15356.8,20084,22956,27043.33333333333,34545,3020144560090947


# Evaluation de la population dans la zone d'étude

In [50]:
# Déplacement des personnes en NPdC en tant qu'origine ou destination (intra-NPdC & origine ou destination HdF)
PECH_personsXY = travelDiary_npdcXY['PECH'].astype(str)

In [51]:
j=0
for PECH in list(set(PECH_personsXY)):
    j=j+1
print('-------> Nbr persons qui se déplacent en NPdC avec soit une origine ou une destination en HdF : used for demand')
print(j)

-------> Nbr persons qui se déplacent en NPdC avec soit une origine ou une destination en HdF : used for demand
52763


In [52]:
print('=======> Verify occurency from data ID')
occurencePECH = travelDiary_npdcXY.PECH.value_counts()

=======> Verify occurency from data ID


In [53]:
occurencePECH

13201202600407502    29
06200602402213201    29
11200900002050901    25
11200900003031701    25
01200400051405502    23
                     ..
02200800060105203     1
08201610200406901     1
02200800010423901     1
14201400600534102     1
08201613601104801     1
Name: PECH, Length: 52763, dtype: int64

# Fin du code

# Test extra

# Déplacements des personnes éligibles

In [8]:
df = pd.read_csv("output_hdf/travelDiaries/travelDiaryDeciles_npdcXY.csv", sep=";",encoding='utf8', dtype=str)

In [9]:
df

,DECH,PECH,COEQ,DP2,D4A,D4B,D3,D3X,D3Y,D2AA,...,Q3_Q1,D115,D215,D315,D415,D615,D715,D815,D915,ECH_x
0,0120040001010040101,01200400010100401,36.8921544,62119,09,45,62119,674661.80202,7048040.28762,01,...,11899.166666666666,8858,10837,12729.62962962963,14623,19225.416666666668,21900.666666666668,25877.33333333333,33081.333333333336,012004000101004
1,0120040001010040102,01200400010100401,36.8921544,62119,10,30,62119,674661.80202,7048040.28762,06,...,11899.166666666666,8858,10837,12729.62962962963,14623,19225.416666666668,21900.666666666668,25877.33333333333,33081.333333333336,012004000101004
2,0120040001010040201,01200400010100402,36.8921544,62119,09,45,62119,674661.80202,7048040.28762,01,...,11899.166666666666,8858,10837,12729.62962962963,14623,19225.416666666668,21900.666666666668,25877.33333333333,33081.333333333336,012004000101004
3,0120040001010040202,01200400010100402,36.8921544,62119,10,30,62119,674661.80202,7048040.28762,06,...,11899.166666666666,8858,10837,12729.62962962963,14623,19225.416666666668,21900.666666666668,25877.33333333333,33081.333333333336,012004000101004
4,0120040001010040203,01200400010100402,36.8921544,62119,10,45,62119,674661.80202,7048040.28762,01,...,11899.166666666666,8858,10837,12729.62962962963,14623,19225.416666666668,21900.666666666668,25877.33333333333,33081.333333333336,012004000101004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233380,30201445600633260102,302014456006332601,9.42639200,80021,16,15,62447,631483.46449,7031127.49468,02,...,12510.304347826086,8911.724137931034,11167.560975609756,13250.5,15356.8,20084,22956,27043.33333333333,34545,3020144560063326
233381,30201445600909400105,302014456009094001,8.06854600,80021,11,00,80021,649001.84505,6978182.20329,01,...,12510.304347826086,8911.724137931034,11167.560975609756,13250.5,15356.8,20084,22956,27043.33333333333,34545,3020144560090940
233382,30201445600909400106,302014456009094001,8.06854600,80021,19,30,62663,660095.34981,7007901.02731,09,...,12510.304347826086,8911.724137931034,11167.560975609756,13250.5,15356.8,20084,22956,27043.33333333333,34545,3020144560090940
233383,30201445600909470101,302014456009094701,8.94751700,80021,05,30,80021,649001.84505,6978182.20329,01,...,12510.304347826086,8911.724137931034,11167.560975609756,13250.5,15356.8,20084,22956,27043.33333333333,34545,3020144560090947


In [22]:
# Extraction des données d'éligibilité : Grands Mobiles (personnes de 11 ans et plus avec un déplacement au moins de 10 km)
##ménages éligibles
is_El = df['MTDQ'] == '1'
df_El = df[is_El]

In [23]:
df_El

,DECH,PECH,COEQ,DP2,D4A,D4B,D3,D3X,D3Y,D2AA,...,Q3_Q1,D115,D215,D315,D415,D615,D715,D815,D915,ECH_x
27,0120040001010600101,01200400010106001,46.8923743,62119,07,45,62119,674661.80202,7048040.28762,01,...,11899.166666666666,8858,10837,12729.62962962963,14623,19225.416666666668,21900.666666666668,25877.33333333333,33081.333333333336,012004000101060
28,0120040001010600102,01200400010106001,46.8923743,62119,08,05,62119,674661.80202,7048040.28762,10,...,11899.166666666666,8858,10837,12729.62962962963,14623,19225.416666666668,21900.666666666668,25877.33333333333,33081.333333333336,012004000101060
29,0120040001010600103,01200400010106001,46.8923743,62119,08,15,62841,672092.80146,7049926.10978,06,...,11899.166666666666,8858,10837,12729.62962962963,14623,19225.416666666668,21900.666666666668,25877.33333333333,33081.333333333336,012004000101060
30,0120040001010600104,01200400010106001,46.8923743,62119,09,00,62119,674661.80202,7048040.28762,01,...,11899.166666666666,8858,10837,12729.62962962963,14623,19225.416666666668,21900.666666666668,25877.33333333333,33081.333333333336,012004000101060
31,0120040001010600105,01200400010106001,46.8923743,62119,10,00,62119,674661.80202,7048040.28762,08,...,11899.166666666666,8858,10837,12729.62962962963,14623,19225.416666666668,21900.666666666668,25877.33333333333,33081.333333333336,012004000101060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233380,30201445600633260102,302014456006332601,9.42639200,80021,16,15,62447,631483.46449,7031127.49468,02,...,12510.304347826086,8911.724137931034,11167.560975609756,13250.5,15356.8,20084,22956,27043.33333333333,34545,3020144560063326
233381,30201445600909400105,302014456009094001,8.06854600,80021,11,00,80021,649001.84505,6978182.20329,01,...,12510.304347826086,8911.724137931034,11167.560975609756,13250.5,15356.8,20084,22956,27043.33333333333,34545,3020144560090940
233382,30201445600909400106,302014456009094001,8.06854600,80021,19,30,62663,660095.34981,7007901.02731,09,...,12510.304347826086,8911.724137931034,11167.560975609756,13250.5,15356.8,20084,22956,27043.33333333333,34545,3020144560090940
233383,30201445600909470101,302014456009094701,8.94751700,80021,05,30,80021,649001.84505,6978182.20329,01,...,12510.304347826086,8911.724137931034,11167.560975609756,13250.5,15356.8,20084,22956,27043.33333333333,34545,3020144560090947


In [24]:
is_Elp = df_El['PTDQ'] == '1'
df_Elp = df_El[is_Elp]

In [25]:
df_Elp

,DECH,PECH,COEQ,DP2,D4A,D4B,D3,D3X,D3Y,D2AA,...,Q3_Q1,D115,D215,D315,D415,D615,D715,D815,D915,ECH_x
34,0120040001010600201,01200400010106002,49.5802118,62119,07,30,62119,674661.80202,7048040.28762,01,...,11899.166666666666,8858,10837,12729.62962962963,14623,19225.416666666668,21900.666666666668,25877.33333333333,33081.333333333336,012004000101060
35,0120040001010600202,01200400010106002,49.5802118,62119,17,45,59350,703330.96289,7059422.55269,02,...,11899.166666666666,8858,10837,12729.62962962963,14623,19225.416666666668,21900.666666666668,25877.33333333333,33081.333333333336,012004000101060
36,0120040001010680101,01200400010106801,47.0660345,62119,09,30,62119,674661.80202,7048040.28762,01,...,11899.166666666666,8858,10837,12729.62962962963,14623,19225.416666666668,21900.666666666668,25877.33333333333,33081.333333333336,012004000101068
37,0120040001010680102,01200400010106801,47.0660345,62119,18,00,62489,666802.33275,7046998.08014,08,...,11899.166666666666,8858,10837,12729.62962962963,14623,19225.416666666668,21900.666666666668,25877.33333333333,33081.333333333336,012004000101068
42,0120040001011610102,01200400010116101,52.8982285,62119,16,00,62119,674661.80202,7048040.28762,01,...,11899.166666666666,8858,10837,12729.62962962963,14623,19225.416666666668,21900.666666666668,25877.33333333333,33081.333333333336,012004000101161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233380,30201445600633260102,302014456006332601,9.42639200,80021,16,15,62447,631483.46449,7031127.49468,02,...,12510.304347826086,8911.724137931034,11167.560975609756,13250.5,15356.8,20084,22956,27043.33333333333,34545,3020144560063326
233381,30201445600909400105,302014456009094001,8.06854600,80021,11,00,80021,649001.84505,6978182.20329,01,...,12510.304347826086,8911.724137931034,11167.560975609756,13250.5,15356.8,20084,22956,27043.33333333333,34545,3020144560090940
233382,30201445600909400106,302014456009094001,8.06854600,80021,19,30,62663,660095.34981,7007901.02731,09,...,12510.304347826086,8911.724137931034,11167.560975609756,13250.5,15356.8,20084,22956,27043.33333333333,34545,3020144560090940
233383,30201445600909470101,302014456009094701,8.94751700,80021,05,30,80021,649001.84505,6978182.20329,01,...,12510.304347826086,8911.724137931034,11167.560975609756,13250.5,15356.8,20084,22956,27043.33333333333,34545,3020144560090947


In [ ]:
menageEligible_hdf = menage_hdf[is_menEl]
##personnes éligibles
is_persEl = person_hdf['PTDQ'] == '1'
personEligible_hdf = person_hdf[is_persEl]

In [34]:
# Déplacement des personnes en Hdf en tant qu'origine ou destination (Hdf & hors périmètre HdF)
df1 = pd.read_csv("output_hdf/travelDiaries/travelDiary_npdc_old.txt",sep=";").astype(str)
PECH_persons1 = df1['PECH'].astype(str)
# Déplacement des personnes en Hdf en tant qu'origine et destination (intra-HdF)
df2 = pd.read_csv("output_hdf/travelDiaries/travelDiary_npdcXY.txt",sep=";").astype(str)
PECH_persons2 = df2['PECH'].astype(str)

C:\Users\Utilisateur\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Utilisateur\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [35]:
i=0
for id_person in list(set(PECH_persons1)):
    i=i+1
print('-------> Nbr persons éligibles dans la base (HdF et de létranger)')
print(i)

j=0
for id_person in list(set(PECH_persons2)):
    j=j+1
print('-------> Nbr persons éligibles qui se déplacent en intra-HdF')
print(j)

-------> Nbr persons éligibles dans la base (HdF et de létranger)
22479
-------> Nbr persons éligibles qui se déplacent en intra-HdF
52763


In [48]:
personOff = pd.read_csv("output_hdf/demand/listPersonOff.txt",sep=";", dtype=str).astype(str)

In [49]:
personOff

,PECH
0,09200401115027403
1,09200401115027403
2,09200401115027403
3,09200401115027404
4,09200401115027404
...,...
4053,14201401300332903
4054,14201401300332903
4055,14201401300332903
4056,14201401300332903


In [50]:
occurencePECH = personOff.PECH.value_counts()

In [51]:
occurencePECH

13201201800105803    14
13201203200812703    13
09200405306061403    13
13201203200812704    12
11200900010200703    12
                     ..
13201202900210104     1
09200405311076303     1
13201203000411403     1
12201002800411003     1
13201200302002603     1
Name: PECH, Length: 1117, dtype: int64

In [5]:
tt = pd.read_csv("tt",sep=";", dtype=str).astype(str)

In [6]:
tt

,PECH_MK
0,01200400010112901
1,01200400010116101
2,01200400010116102
3,01200400010116103
4,01200400010116104
...,...
52771,302014456005291601
52772,302014456006034302
52773,302014456006332601
52774,302014456009094001


In [8]:
occurencePECH = tt.PECH_MK.value_counts()

In [9]:
occurencePECH

08201610700305101     2
14201401100106501     2
11200900008087703     2
14201401100100202     2
11200900008072901     2
                     ..
302014103016123602    1
302014151001064201    1
13201201500903502     1
302014404020076601    1
09200417115020501     1
Name: PECH_MK, Length: 52763, dtype: int64